### Data preprocessing

In [2]:
import numpy as np
import pandas as pd
import matplotlib as plt

In [3]:
trainingData = pd.read_csv('../../data/trainData.csv', header=None)
# AATI = Average Access Time Interval
trainingData.columns = ["timestamp", "sector_id", "# of blocks"]
trainingData.head()

# trainingData.Frequency.hist(bins=15)

,timestamp,sector_id,# of blocks
0,1.652817,7487488,2048
1,1.652824,7489536,2048
2,1.652830,7491584,2048
3,1.652836,7493632,2048
4,1.652842,7495680,2048


In [4]:
testData = pd.read_csv("../../data/testData.csv", header=None)
testData.columns = ["timestamp", "sector_id", "# of blocks"]
testData.head()

print(testData.head())

   timestamp  sector_id  # of blocks
0   0.000000     303567            7
1   0.000000      55590            6
2   0.026214     303574            7
3   0.026214     240840            6
4   0.117964     303581            7


In [5]:
testDataLabel = pd.read_csv("../../data/lableing/testDataLabeled.csv", header=None)
testDataLabel.columns = ["sector_id", "frequency", "AATI", "# of blocks","hot/cold"]
print(testDataLabel.head())

   sector_id  frequency      AATI  # of blocks  hot/cold
0     753921      90736  0.459198       544416         1
1     240840      48735  0.886414       292465         1
2     836706      31787  1.296780       195293         1
3     837306      31704  1.299350       192217         1
4     700132      31288  1.156710       247313         1


In [6]:
trainDataLabel = pd.read_csv("../../data/lableing/trainDataLabeled.csv", header=None)
trainDataLabel.columns = ["sector_id", "frequency", "AATI", "# of blocks","hot/cold"]
print(trainDataLabel.head())

   sector_id  frequency     AATI  # of blocks  hot/cold
0       8488        966  36.0808         7728         1
1     205888        948  36.7295         7584         1
2     206064        948  36.7666         7584         1
3      74328        947  36.7683         7576         1
4      74408        945  36.8834         7560         1


In [7]:
# Constants
trainLabelSize = trainDataLabel["sector_id"].size
print("trainDataLabel size:", trainLabelSize)

trainDataLabel size: 1586700


In [8]:
trainDataLabel.dtypes

sector_id        int64
frequency        int64
AATI           float64
# of blocks      int64
hot/cold         int64
dtype: object

In [9]:
trainingData.dtypes

timestamp      float64
sector_id        int64
# of blocks      int64
dtype: object

In [10]:
trainingYLabelMap = {}

for i in range(trainLabelSize):
    trainingYLabelMap[trainDataLabel["sector_id"][i]] = trainDataLabel["hot/cold"][i]

trainingData["hot/cold"] = [trainingYLabelMap[sectorId] if sectorId in trainingYLabelMap else np.nan for sectorId in trainingData["sector_id"]]


In [22]:
testLabelSize = testDataLabel["sector_id"].size
testYLabelMap = {}

for i in range(testLabelSize):
    testYLabelMap[testDataLabel["sector_id"][i]] = testDataLabel["hot/cold"][i]

testData["hot/cold"] = [testYLabelMap[sectorId] if sectorId in testYLabelMap else np.nan for sectorId in testData["sector_id"]]

In [11]:
trainDataLabel["hot/cold"].value_counts()

0    1354099
1     232601
Name: hot/cold, dtype: int64

In [12]:
trainingData["hot/cold"].value_counts()


1    6057980
0    3628281
Name: hot/cold, dtype: int64

In [23]:
testData["hot/cold"].value_counts()

1    2070048
0    2029306
Name: hot/cold, dtype: int64

In [13]:
# Normalizing number of blocks
trainBlocksMean = trainingData["# of blocks"].mean()
trainBlocksStd = trainingData["# of blocks"].std()

trainingData["# of blocks"] = (trainingData["# of blocks"] - trainBlocksMean) / trainBlocksStd

testBlocksMean = testData["# of blocks"].mean()
testBlocksStd = testData["# of blocks"].std()

testData["# of blocks"] = (testData["# of blocks"] - testBlocksMean) / testBlocksStd

In [14]:
import seaborn as sns

In [17]:
# Tried to plot the normalized data, ran into some kind of error . . .
plt.figure(figsize=(12,6))
ax = sns.violinplot(x='Column', y='Normalized', data=trainingData["# of blocks"])

TypeError: 'module' object is not callable

In [14]:
features = trainingData[["sector_id","# of blocks"]].to_numpy().tolist()
target = trainingData[["hot/cold"]].to_numpy().tolist()



In [15]:
features[0:5]


[[7487488.0, 0.6759364762275851],
 [7489536.0, 0.6759364762275851],
 [7491584.0, 0.6759364762275851],
 [7493632.0, 0.6759364762275851],
 [7495680.0, 0.6759364762275851]]

In [16]:
target[0:11]

[[0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0]]

### Model Implementation

In [17]:
import tensorflow as tf

In [18]:
maxSectorNumber = np.amax(trainingData["sector_id"])
maxBlocks = np.amax(trainingData["# of blocks"])
print("maxSectorNumber:", maxSectorNumber)
print("maxBlocks:", maxBlocks)

maxSectorNumber: 1000213824
maxBlocks: 0.6759364762275851


In [20]:
with tf.device('/gpu:1'):
    embeddingLayer = tf.keras.layers.Embedding(input_dim=maxSectorNumber+1, output_dim=64)
    trainingData["sector_id"] = embeddingLayer(trainingData["sector_id"])

2022-07-24 22:30:27.768337: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 256054739200 exceeds 10% of free system memory.
2022-07-24 22:30:27.768603: W tensorflow/core/framework/op_kernel.cc:1745] OP_REQUIRES failed at random_op.cc:74 : RESOURCE_EXHAUSTED: OOM when allocating tensor with shape[1000213825,64] and type float on /job:localhost/replica:0/task:0/device:CPU:0 by allocator cpu


ResourceExhaustedError: OOM when allocating tensor with shape[1000213825,64] and type float on /job:localhost/replica:0/task:0/device:CPU:0 by allocator cpu [Op:RandomUniform]

In [34]:
model = Sequential()

# embedding the sector number from 0-1
model.add(tf.keras.layers.Embedding(input_dim=maxSectorNumber+1,output_dim=64))

model.add(tf.keras.layers.LSTM)

2022-07-22 01:29:14.219727: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-22 01:29:14.220148: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-07-22 01:29:14.220236: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2022-07-22 01:29:14.220299: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2022-07-22 01:29:14.220364: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Co